meant for modified xgbmodel

In [8]:
import optuna
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [9]:
#import cleaned data set
df = pd.read_csv("clean_dataset.csv")

In [10]:
#hide warnings so they do not affect the functionality of the package
import warnings
warnings.filterwarnings('ignore')

In [11]:
features_list = ['Maritalstatus', 'Applicationmode', 'Applicationorder', 'Course',
       'Daytimeeveningattendance', 'Previousqualification', 'Nationality',
       'Mothersqualification', 'Fathersqualification', 'Mothersoccupation',
       'Fathersoccupation', 'Displaced', 'Educationalspecialneeds', 'Debtor',
       'Tuitionfeesuptodate', 'Gender', 'Scholarshipholder', 'Ageatenrollment',
       'International', 'Curricularunits1stsem(credited)',
       'Curricularunits1stsem(enrolled)', 'Curricularunits1stsem(evaluations)',
       'Curricularunits1stsem(approved)', 'Curricularunits1stsem(grade)',
       'Curricularunits1stsem(withoutevaluations)',
       'Curricularunits2ndsem(credited)', 'Curricularunits2ndsem(enrolled)',
       'Curricularunits2ndsem(evaluations)', 'Curricularunits2ndsem(approved)',
       'Curricularunits2ndsem(grade)',
       'Curricularunits2ndsem(withoutevaluations)', 'Unemploymentrate',
       'Inflationrate', 'GDP']

forecast_var = ["Target"]

In [12]:
from sklearn.preprocessing import LabelEncoder

X = df[features_list]
y = df[forecast_var] 

# Encode categorical features
y = y.astype("category")
le = LabelEncoder()
y = le.fit_transform(np.ravel(y))

#Normalise numerical features
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9)

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

def objective(trial):
    #define the hyperparameters to optimize
    param = {
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 10000, 100),
        'eta': trial.suggest_discrete_uniform('eta', 0.01, 0.1, 0.01),
        'reg_alpha': trial.suggest_int('reg_alpha', 1, 50),
        'reg_lambda': trial.suggest_int('reg_lambda', 5, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0)}

    #define model
    
    #model = pd.read_json('model_3.json')
    model = xgb.XGBClassifier(n_estimators = 3900, early_stopping_rounds = 50, learning_rate = 0.001, max_depth = 12, tree_method = 'approx',booster = 'dart',enable_categorical=True)
    model.fit(X_train, y_train, eval_set = [(X_test,y_test)], verbose = 10) 
    
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    return acc

In [14]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20) #should be more
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-03-27 22:56:52,148] A new study created in memory with name: no-name-9583f199-16e2-4ab8-a8ed-87cab7706e80
[W 2023-03-27 22:56:52,159] Trial 0 failed with parameters: {'max_depth': 12, 'subsample': 0.65, 'n_estimators': 6700, 'eta': 0.06999999999999999, 'reg_alpha': 16, 'reg_lambda': 61, 'min_child_weight': 10, 'colsample_bytree': 0.5225612418703874} because of the following error: ValueError('Expected object or value').
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/hv/6z0_223n04z97twbr2tkzfqw0000gn/T/ipykernel_19098/253437437.py", line 17, in objective
    model = pd.read_json('model_3.json')
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/util/_decorators.py", line 207, in wrapper
    return func(*args, **kwargs)
  File "/Library/Frameworks/Python.fr

ValueError: Expected object or value

In [ ]:
#get the best parameters and train and evaluate the RFC model with the best parameters
best_params = study.best_params
model = xgb.XGBRegressor(**best_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy after tuning: %.2f%%" % (accuracy * 100.0))
print(classification_report(y_test, y_pred))